In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

from dlsvm import utils
from dlsvm import vgg_pytorch as vgg

import torch
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np

import os
import copy

In [2]:
torch.cuda.set_device(1)

### CIFAR-10

In [3]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

batch_size = 128
n_epochs = 1


print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

train_set = datasets.CIFAR10(root='../../RandomNet/data', train=True, download=True, transform=transform_train)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)

test_set = datasets.CIFAR10(root='../../RandomNet/data', train=False, download=True, transform=transform_test)
test_loader = DataLoader(test_set, batch_size=100, shuffle=False, num_workers=2)

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified


In [4]:
def lr_lambda(epoch):
    return (0.5 ** (epoch // 30))

### Usual VGG 16

In [5]:
model = vgg.VGG(vgg.make_layers(vgg.cfg['D'], batch_norm=True), kernel_type=None)
model.cuda()

parameters = [p for p in model.parameters() if p.requires_grad]
optimizer = optim.SGD(parameters, lr=0.05, momentum=0.9, weight_decay=5e-4)
scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda)

n_epochs = 350

utils.train(train_loader, test_loader, model, optimizer, scheduler=scheduler, loss_type='cross_entropy', start_epoch=0, stop_epoch=n_epochs)
utils.save(model, '../checkpoint/vgg16_epoch={}.t7'.format(n_epochs))

Train Epoch: 0 [50000/50000 (100%)]	Loss: 1.710540	Test acc: 0.510


Train Epoch: 1 [50000/50000 (100%)]	Loss: 1.206981	Test acc: 0.644


Train Epoch: 2 [50000/50000 (100%)]	Loss: 0.969176	Test acc: 0.704


Train Epoch: 3 [50000/50000 (100%)]	Loss: 0.830543	Test acc: 0.715


Train Epoch: 4 [50000/50000 (100%)]	Loss: 0.750116	Test acc: 0.748


...

Train Epoch: 345 [50000/50000 (100%)]	Loss: 0.000899	Test acc: 0.938


Train Epoch: 346 [50000/50000 (100%)]	Loss: 0.000936	Test acc: 0.938


Train Epoch: 347 [50000/50000 (100%)]	Loss: 0.000869	Test acc: 0.938


Train Epoch: 348 [50000/50000 (100%)]	Loss: 0.000887	Test acc: 0.937


Train Epoch: 349 [50000/50000 (100%)]	Loss: 0.000865	Test acc: 0.938

Saving..


### VGG16 with svm loss

In [ ]:
kernel_type = 'rbf'
gamma = 1 / 512
n_features = 2048
C = 1

model = vgg.VGG(vgg.make_layers(vgg.cfg['D'], batch_norm=True), kernel_type=kernel_type, n_features=n_features, gamma=gamma)
model.cuda()

parameters = [p for p in model.parameters() if p.requires_grad]
optimizer = optim.Adam(parameters, lr=1e-4, weight_decay=5e-4)

scheduler = lr_scheduler.LambdaLR(optimizer, lambda epoch: 0.5 ** (epoch // 30))

n_epochs = 350

utils.train(train_loader, test_loader, model, optimizer, scheduler=scheduler, loss_type='svml1loss2', C=C, start_epoch=0, stop_epoch=n_epochs)

utils.save(model, '../checkpoint/vgg16_svm_{}_{}_{}_multiclass_loss_epoch={}.t7'.format(kernel_type, n_features, C, n_epochs))

Train Epoch: 0 [50000/50000 (100%)]	Loss: 736.732907	Test acc: 0.476


Train Epoch: 1 [50000/50000 (100%)]	Loss: 424.175946	Test acc: 0.547


Train Epoch: 2 [50000/50000 (100%)]	Loss: 317.152292	Test acc: 0.599


Train Epoch: 3 [50000/50000 (100%)]	Loss: 257.914411	Test acc: 0.670


Train Epoch: 4 [50000/50000 (100%)]	Loss: 219.576427	Test acc: 0.680


...

Train Epoch: 156 [50000/50000 (100%)]	Loss: 5.757672	Test acc: 0.885


Train Epoch: 157 [50000/50000 (100%)]	Loss: 5.674811	Test acc: 0.886


Train Epoch: 158 [50000/50000 (100%)]	Loss: 5.808954	Test acc: 0.885


Train Epoch: 159 [50000/50000 (100%)]	Loss: 5.836257	Test acc: 0.884


Train Epoch: 160 [50000/50000 (100%)]	Loss: 5.715834	Test acc: 0.884


In [5]:
utils.evaluate(model, test_loader)

0.8871